### Merging 2019 Weather Data and Initial Inspection and Cleaning

In [1]:
import os
import pandas as pd
file_list = os.listdir()
flat_file_list = list(filter(lambda x: x.endswith('csv'), file_list)) 
data_frames = []
for file_name in flat_file_list:
    df = pd.read_csv(file_name)
    data_frames.append(df)
merged_frame = pd.concat(data_frames)

In [2]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,24,15.5,24,...,18.5,31.5,30.2,*,17.4,,0.00,G,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,24,31.3,24,...,33.0,43.5,44.1,,28.2,,0.00,G,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,24,31.6,24,...,29.9,46.4,39.2,*,33.6,,0.00,G,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,24,32.5,24,...,22.1,42.7,41.0,,33.6,,0.29,G,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,24,34.6,24,...,27.6,39.8,38.1,*,34.2,*,0.00,G,999.9,101000


There appear to be zero and NaN latitude/longitude data.  For now lets verify that we have nonempty entries in these fields.  I can always lookup by station number later if I need to find data near a point.

In [3]:
merged_frame[(merged_frame['LATITUDE'] != 0)].dropna().head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,24,15.5,24,...,18.5,31.5,30.2,*,17.4,,0.00,G,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,24,31.3,24,...,33.0,43.5,44.1,,28.2,,0.00,G,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,24,31.6,24,...,29.9,46.4,39.2,*,33.6,,0.00,G,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,24,32.5,24,...,22.1,42.7,41.0,,33.6,,0.29,G,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,24,34.6,24,...,27.6,39.8,38.1,*,34.2,*,0.00,G,999.9,101000


There are entries with valid latitude and longitude.  Table will be tidied up and then exported to a merged CSV for DB import.

Temp attributes give the number of measurements averaged for mean temperature.  This will not assist me in modeling, so I am going to drop this column.

In [4]:
merged_frame.drop(merged_frame.columns[7], axis = 1, inplace = True)

In [5]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,DEWP_ATTRIBUTES,SLP,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,24,1025.1,...,18.5,31.5,30.2,*,17.4,,0.00,G,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,24,1017.0,...,33.0,43.5,44.1,,28.2,,0.00,G,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,24,1009.0,...,29.9,46.4,39.2,*,33.6,,0.00,G,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,24,1011.3,...,22.1,42.7,41.0,,33.6,,0.29,G,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,24,1007.7,...,27.6,39.8,38.1,*,34.2,*,0.00,G,999.9,101000


The same situation applies with the other attribute fields.  Removing them as well.

In [6]:
merged_frame.drop(['DEWP_ATTRIBUTES', 'MAX_ATTRIBUTES', 'MIN_ATTRIBUTES', 'PRCP_ATTRIBUTES'], axis = 1, inplace=True)

In [7]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,24,...,6,8.9,24,18.5,31.5,30.2,17.4,0.00,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,24,...,6,17.4,23,33.0,43.5,44.1,28.2,0.00,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,24,...,6,11.0,24,29.9,46.4,39.2,33.6,0.00,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,24,...,6,7.7,24,22.1,42.7,41.0,33.6,0.29,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,24,...,6,7.0,24,27.6,39.8,38.1,34.2,0.00,999.9,101000


For Temp, dewpoint, SLP, and STP missing values are reported as 9999.9.  These are going to be replaced with NaN for clarity and DB processing purposes.

In [8]:
import numpy as np

def set_null_9999(arg):
    if arg == 9999.9:
        return np.NaN
    else:
        return arg
    
merged_frame['TEMP'] = merged_frame['TEMP'].apply(set_null_9999) 

In [9]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,24,...,6,8.9,24,18.5,31.5,30.2,17.4,0.00,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,24,...,6,17.4,23,33.0,43.5,44.1,28.2,0.00,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,24,...,6,11.0,24,29.9,46.4,39.2,33.6,0.00,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,24,...,6,7.7,24,22.1,42.7,41.0,33.6,0.29,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,24,...,6,7.0,24,27.6,39.8,38.1,34.2,0.00,999.9,101000


In [10]:
merged_frame['DEWP'] = merged_frame['DEWP'].apply(set_null_9999)
merged_frame['SLP'] = merged_frame['SLP'].apply(set_null_9999) 
merged_frame['STP'] = merged_frame['STP'].apply(set_null_9999)

In [11]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,24,...,6,8.9,24,18.5,31.5,30.2,17.4,0.00,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,24,...,6,17.4,23,33.0,43.5,44.1,28.2,0.00,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,24,...,6,11.0,24,29.9,46.4,39.2,33.6,0.00,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,24,...,6,7.7,24,22.1,42.7,41.0,33.6,0.29,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,24,...,6,7.0,24,27.6,39.8,38.1,34.2,0.00,999.9,101000


For VISIB, WDSP, MXSPD, and GUST the missing value is encoded as 999.9.  As before replacing with NaN.

In [12]:
def set_null_999(arg):
    if arg == 999.9:
        return np.NaN
    else:
        return arg
    
merged_frame['VISIB'] = merged_frame['VISIB'].apply(set_null_999) 

In [13]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,24,...,6,8.9,24,18.5,31.5,30.2,17.4,0.00,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,24,...,6,17.4,23,33.0,43.5,44.1,28.2,0.00,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,24,...,6,11.0,24,29.9,46.4,39.2,33.6,0.00,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,24,...,6,7.7,24,22.1,42.7,41.0,33.6,0.29,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,24,...,6,7.0,24,27.6,39.8,38.1,34.2,0.00,999.9,101000


In [14]:
merged_frame['WDSP'] = merged_frame['WDSP'].apply(set_null_999)
merged_frame['MXSPD'] = merged_frame['MXSPD'].apply(set_null_999)
merged_frame['GUST'] = merged_frame['GUST'].apply(set_null_999)

In [15]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,SLP_ATTRIBUTES,...,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,24,...,6,8.9,24,18.5,31.5,30.2,17.4,0.00,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,24,...,6,17.4,23,33.0,43.5,44.1,28.2,0.00,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,24,...,6,11.0,24,29.9,46.4,39.2,33.6,0.00,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,24,...,6,7.7,24,22.1,42.7,41.0,33.6,0.29,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,24,...,6,7.0,24,27.6,39.8,38.1,34.2,0.00,999.9,101000


In [16]:
merged_frame.drop(['SLP_ATTRIBUTES', 'VISIB_ATTRIBUTES', 'WDSP_ATTRIBUTES'], axis=1, inplace=True)

In [17]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,STP_ATTRIBUTES,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,23.9,24,21.2,8.9,18.5,31.5,30.2,17.4,0.00,999.9,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,15.8,24,12.5,17.4,33.0,43.5,44.1,28.2,0.00,999.9,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,7.8,24,10.8,11.0,29.9,46.4,39.2,33.6,0.00,999.9,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,10.1,24,3.8,7.7,22.1,42.7,41.0,33.6,0.29,999.9,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,6.6,24,3.3,7.0,27.6,39.8,38.1,34.2,0.00,999.9,101000


For snow depth missing data will be reported as 999.9.  However, most stations do not report 0 on days with no snow on ground.  As such I will clean this value to zero on such days.

In [18]:
def set_zero_999(arg):
    if arg == 999.9:
        return 0
    else:
        return arg
    
merged_frame['SNDP'] = merged_frame['SNDP'].apply(set_zero_999)

In [19]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,STP_ATTRIBUTES,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,23.9,24,21.2,8.9,18.5,31.5,30.2,17.4,0.00,0.0,1000
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,15.8,24,12.5,17.4,33.0,43.5,44.1,28.2,0.00,0.0,10000
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,7.8,24,10.8,11.0,29.9,46.4,39.2,33.6,0.00,0.0,10000
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,10.1,24,3.8,7.7,22.1,42.7,41.0,33.6,0.29,0.0,100000
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,6.6,24,3.3,7.0,27.6,39.8,38.1,34.2,0.00,0.0,101000


FRSHTT is a combined indicator for occurences of meteorological conditions.  The interpretation for this field is coppied from the data source readme (https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/readme.txt).


FRSHTT - Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day of:
                         Fog ('F' - 1st digit).
                         Rain or Drizzle ('R' - 2nd digit).
                         Snow or Ice Pellets ('S' - 3rd digit).
                         Hail ('H' - 4th digit).
                         Thunder ('T' - 5th digit).
                         Tornado or Funnel Cloud ('T' - 6th digit).

In [20]:
def process_FRSHTT(arg):
    string = str(arg)
    if len(string)>6:
        #more than six chars would be invalid
        return np.NaN
    index = 0
    outputs = ['Fog', 'Rain', 'Snow', 'Hail', 'Thunder', 'Tornado']
    output = []
    for char in string:
        if char == '1':
            output.append(outputs[index])
        index+=1
    return "|".join(output)

In [21]:
merged_frame['FRSHTT'] = merged_frame['FRSHTT'].apply(process_FRSHTT)

In [22]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,STP_ATTRIBUTES,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,23.9,24,21.2,8.9,18.5,31.5,30.2,17.4,0.00,0.0,Fog
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,15.8,24,12.5,17.4,33.0,43.5,44.1,28.2,0.00,0.0,Fog
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,7.8,24,10.8,11.0,29.9,46.4,39.2,33.6,0.00,0.0,Fog
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,10.1,24,3.8,7.7,22.1,42.7,41.0,33.6,0.29,0.0,Fog
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,6.6,24,3.3,7.0,27.6,39.8,38.1,34.2,0.00,0.0,Fog|Snow


In [23]:
merged_frame.drop('STP_ATTRIBUTES', axis=1, inplace=True)

99.99 denotes a missing PRCP value.  Many stations do not report precip if the value is 0.  So I will replace those values with 0.

In [24]:
def set_zero_99(arg):
    if arg == 99.99:
        return 0
    else:
        return arg
    
merged_frame['PRCP'] = merged_frame['PRCP'].apply(set_zero_99)

In [25]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,23.9,21.2,8.9,18.5,31.5,30.2,17.4,0.00,0.0,Fog
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,15.8,12.5,17.4,33.0,43.5,44.1,28.2,0.00,0.0,Fog
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,7.8,10.8,11.0,29.9,46.4,39.2,33.6,0.00,0.0,Fog
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,10.1,3.8,7.7,22.1,42.7,41.0,33.6,0.29,0.0,Fog
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,6.6,3.3,7.0,27.6,39.8,38.1,34.2,0.00,0.0,Fog|Snow


MAX and MIN also report missing data as 9999.9.  Cleaning as such with NaN.

In [26]:
merged_frame['MAX'] = merged_frame['MAX'].apply(set_null_9999) 
merged_frame['MIN'] = merged_frame['MIN'].apply(set_null_9999)

In [27]:
merged_frame.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,1001099999,2019-01-01,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",24.0,15.5,1025.1,23.9,21.2,8.9,18.5,31.5,30.2,17.4,0.00,0.0,Fog
1,1001099999,2019-01-02,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",36.2,31.3,1017.0,15.8,12.5,17.4,33.0,43.5,44.1,28.2,0.00,0.0,Fog
2,1001099999,2019-01-03,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.5,31.6,1009.0,7.8,10.8,11.0,29.9,46.4,39.2,33.6,0.00,0.0,Fog
3,1001099999,2019-01-04,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,32.5,1011.3,10.1,3.8,7.7,22.1,42.7,41.0,33.6,0.29,0.0,Fog
4,1001099999,2019-01-05,70.933333,-8.666667,9.0,"JAN MAYEN NOR NAVY, NO",35.8,34.6,1007.7,6.6,3.3,7.0,27.6,39.8,38.1,34.2,0.00,0.0,Fog|Snow


In [28]:
merged_frame.to_csv('2019-merged-cleaned.csv.dat')#appending .dat to avoid accidental reprocessing if script is rerun.
